In [4]:
import pandas as pd

# Kostenstruktur
kostenarten = [
    ('Entwicklungskosten', [ 
        ('Softwareentwicklung (App/Dashboard für Entsorger)', 90000.0),
        ('Sonstige Entwicklungskosten (z.B. Testing, Debugging, Schnittstellen, UI)', 90000.0),
        ('Technische Entwicklung LoRaWAN (Integration, Kommunikation)', 40000.0),
    ]),

    ('Materialkosten', [
        ('240L Mülltonne (Basis)', 28.0),
        ('Arduino NodeMCU', 3.5),
        ('Ultraschall-Sensor (HCSR04)', 0.4),
        ('Neigungssensor', 1.0),
        ('Gewichtssensor', 1.9),
        ('Temperatursensor', 1.0),
        ('GPS-Modul', 4.2),
        ('RFID-Reader', 0.85),
        ('LoRaWAN-Modul', 0.52),
        ('Solarpanel', 3.9),
        ('Akku', 1.5),
        ('Gehäuse', 1.0),
        ('Montagekosten', 40.0),
        ('Zubehör (Kabel, Steckpins, ...)', 0.2)
    ]),

    ('Betriebskosten', [
        ('Netzwerk-Betriebskosten (jährlich, geschätzt)', 30.0),
        ('Sonstiges (z.B. Verpackung, Logistik, Wartung)', 20.0)  
    ])
]

anzahl_tonnen = 1000
anzahl_entsorger  = 5
nutzungsdauer = 3  # Jahre

# DataFrame erstellen
daten = []
for kategorie, komponenten in kostenarten:
    for name, preis in komponenten:
        daten.append({'Kostenart': kategorie, 'Komponente': name, 'Stückpreis_EUR': preis})

kosten_df = pd.DataFrame(daten)

# Produktionskosten berechnen
kosten_df['Gesamtpreis_EUR'] = kosten_df.apply(
    lambda row: row['Stückpreis_EUR'] * anzahl_tonnen if row['Kostenart'] in ['Materialkosten', 'Betriebskosten'] else row['Stückpreis_EUR'],
    axis=1
)

# Gesamtkosten pro Kategorie
gesamt_pro_kategorie = kosten_df.groupby('Kostenart')['Gesamtpreis_EUR'].sum().to_dict()
gesamtkosten_alle_tonnen = kosten_df['Gesamtpreis_EUR'].sum()

# Einmalige Entwicklungskosten
einmalige_entwicklungskosten = gesamt_pro_kategorie.get('Entwicklungskosten', 0)

# Laufende Kosten pro Tonne (jährlich)
laufende_kosten_pro_tonne = (gesamt_pro_kategorie.get('Materialkosten', 0) + gesamt_pro_kategorie.get('Betriebskosten', 0)) / anzahl_tonnen

# Verkauf (Einmaliger Erlös, keine jährlichen Einnahmen aus Tonnen-Verkauf)
verkaufspreis_pro_tonne = laufende_kosten_pro_tonne * 1.6
gewinn_pro_tonne = verkaufspreis_pro_tonne - laufende_kosten_pro_tonne
gewinn_aus_verkauf = gewinn_pro_tonne * anzahl_tonnen  # Einmalig im Jahr 1

# Leasingmodell (jährliche Leasingrate basierend auf Kosten / Nutzungsdauer)
leasing_einnahmen_pro_jahr = (laufende_kosten_pro_tonne / nutzungsdauer) * anzahl_tonnen * 1.6
leasing_kosten_pro_jahr = (laufende_kosten_pro_tonne / nutzungsdauer) * anzahl_tonnen
gewinn_aus_leasing_pro_jahr = leasing_einnahmen_pro_jahr - leasing_kosten_pro_jahr
gewinn_aus_leasing_gesamt = gewinn_aus_leasing_pro_jahr * nutzungsdauer

# Laufende Einnahmen pro Jahr (Service, Premium, Daten)
monatliche_servicegebuehr = 10.0
service_kosten_pro_tonne = 5
quote_service_nutzung = 0.3  # 30% Nutzung

einnahmen_durch_service_pro_jahr = 12 * monatliche_servicegebuehr * anzahl_tonnen * quote_service_nutzung
kosten_service_pro_jahr = service_kosten_pro_tonne * anzahl_tonnen
gewinn_aus_service_pro_jahr = einnahmen_durch_service_pro_jahr - kosten_service_pro_jahr
gewinn_aus_service_gesamt = gewinn_aus_service_pro_jahr * nutzungsdauer

einnahmen_durch_daten_pro_jahr = 4000
einnahmen_durch_datenpartnerschaften_pro_jahr = 8000

monatliche_premium_servicegebuehr_entsorger = 50.0
quote_premium_entsorger = 0.3

einnahmen_durch_premium_entsorger_pro_jahr = monatliche_premium_servicegebuehr_entsorger * 12 * anzahl_entsorger * quote_premium_entsorger
gewinn_aus_premium_entsorger_gesamt = einnahmen_durch_premium_entsorger_pro_jahr * nutzungsdauer

# Gesamtgewinn über 3 Jahre
gesamtgewinn_verkauf = gewinn_aus_verkauf + gewinn_aus_service_gesamt + (einnahmen_durch_daten_pro_jahr + einnahmen_durch_datenpartnerschaften_pro_jahr) * nutzungsdauer + gewinn_aus_premium_entsorger_gesamt
gesamtgewinn_leasing = gewinn_aus_leasing_gesamt + gewinn_aus_service_gesamt + (einnahmen_durch_daten_pro_jahr + einnahmen_durch_datenpartnerschaften_pro_jahr) * nutzungsdauer + gewinn_aus_premium_entsorger_gesamt

# Entwicklungskosten aufschlüsseln
entwicklung_df = kosten_df[kosten_df['Kostenart'] == 'Entwicklungskosten']

# Deckungsbeitrag & Break-Even-Point (pro Jahr betrachtet)
deckungsbeitrag_pro_tonne = verkaufspreis_pro_tonne - laufende_kosten_pro_tonne
gesamtdeckungsbeitrag = deckungsbeitrag_pro_tonne * anzahl_tonnen

break_even_point_tonnen = einmalige_entwicklungskosten / deckungsbeitrag_pro_tonne
break_even_point_tonnen = round(break_even_point_tonnen)

bericht = f"""
=== Kalkulation Smarte Mülltonne ===

Kostenübersicht pro Kategorie:
Entwicklungskosten:
"""
for _, row in entwicklung_df.iterrows():
    bericht += f"- {row['Komponente']}: {row['Gesamtpreis_EUR']:.2f} €\n"

bericht += f"""
Materialkosten (gesamt): {gesamt_pro_kategorie.get('Materialkosten', 0):.2f} €
Betriebskosten (jährlich, gesamt): {gesamt_pro_kategorie.get('Betriebskosten', 0):.2f} €

Einmalige Entwicklungskosten (gesamt): {einmalige_entwicklungskosten:.2f} €
Laufende Kosten pro Tonne (jährlich, ohne Entwicklungskosten): {laufende_kosten_pro_tonne:.2f} €
Verkaufspreis pro Tonne (Einmalverkauf): {verkaufspreis_pro_tonne:.2f} €
Gewinn pro Tonne (Verkauf): {gewinn_pro_tonne:.2f} €
Gewinn aus Verkauf der Tonnen (einmalig): {gewinn_aus_verkauf:.2f} €

Leasingmodell (über {nutzungsdauer} Jahre):
Gewinn aus Leasing pro Jahr: {gewinn_aus_leasing_pro_jahr:.2f} €
Gesamtgewinn aus Leasing: {gewinn_aus_leasing_gesamt:.2f} €

Laufende Einnahmen (über {nutzungsdauer} Jahre):
Gewinn aus Service: {gewinn_aus_service_gesamt:.2f} €
Einnahmen aus Datenverwertung (gesamt): {(einnahmen_durch_daten_pro_jahr + einnahmen_durch_datenpartnerschaften_pro_jahr) * nutzungsdauer:.2f} €
Gewinn aus Premium-Service (Entsorger): {gewinn_aus_premium_entsorger_gesamt:.2f} €

Gesamtgewinn über 3 Jahre (Verkaufsszenario): {gesamtgewinn_verkauf:.2f} €
Gesamtgewinn über 3 Jahre (Leasing-Szenario): {gesamtgewinn_leasing:.2f} €

Deckungsbeitrag pro Tonne: {deckungsbeitrag_pro_tonne:.2f} €
Gesamtdeckungsbeitrag: {gesamtdeckungsbeitrag:.2f} €
Break-Even-Point: {break_even_point_tonnen} Tonnen
"""

print(bericht)



=== Kalkulation Smarte Mülltonne ===

Kostenübersicht pro Kategorie:
Entwicklungskosten:
- Softwareentwicklung (App/Dashboard für Entsorger): 90000.00 €
- Sonstige Entwicklungskosten (z.B. Testing, Debugging, Schnittstellen, UI): 90000.00 €
- Technische Entwicklung LoRaWAN (Integration, Kommunikation): 40000.00 €

Materialkosten (gesamt): 87970.00 €
Betriebskosten (jährlich, gesamt): 50000.00 €

Einmalige Entwicklungskosten (gesamt): 220000.00 €
Laufende Kosten pro Tonne (jährlich, ohne Entwicklungskosten): 137.97 €
Verkaufspreis pro Tonne (Einmalverkauf): 220.75 €
Gewinn pro Tonne (Verkauf): 82.78 €
Gewinn aus Verkauf der Tonnen (einmalig): 82782.00 €

Leasingmodell (über 3 Jahre):
Gewinn aus Leasing pro Jahr: 27594.00 €
Gesamtgewinn aus Leasing: 82782.00 €

Laufende Einnahmen (über 3 Jahre):
Gewinn aus Service: 93000.00 €
Einnahmen aus Datenverwertung (gesamt): 36000.00 €
Gewinn aus Premium-Service (Entsorger): 2700.00 €

Gesamtgewinn über 3 Jahre (Verkaufsszenario): 214482.00 €
Ges